To only run one time at the beginning of the test (unless you want to change the current overlay) :

In [1]:
from pynq import Overlay
ol = Overlay("overlay/test_platform_kairos.bit")

To run only if you made change in the overlay :

In [2]:
ol.download()
# ol?

We define our SPI hardware object : AxiQspi.

In [3]:
from pynq import MMIO

SPI_regSpace = ol.ip_dict['axi_quad_spi_0']
IP_BASE_ADDRESS = SPI_regSpace['phys_addr']
IP_ADDRESS_RNGE = SPI_regSpace['addr_range']
AxiQspi = MMIO(IP_BASE_ADDRESS, IP_ADDRESS_RNGE)

In [4]:
with open("style.css","r") as file:
        css_file = file.read()

Importing some libraries to launch tests

In [ ]:
import os
import package as pkg

from nb import parser, translator
import nb.spi as spi
from nb.reports import builder_html, builder_txt
from nb.reports.content.error_msg import test_config_error        

Tests if the config is correct

In [6]:
if (pkg.single_test_config["activate"] == pkg.all_test_config["activate"]):
    test_config_error()

Open the config file

In [ ]:
spi.cnfg_master_spi(AxiQspi, clk_phase=0, clk_pol=0)

# ALL tests config :
if (pkg.all_test_config["activate"]) :
    all_test_files = os.listdir("./data")
    ignored_test_files = pkg.all_test_config['ignored_test_files']
    list_test_files = [f for f in all_test_files if f not in ignored_test_files]
    for test_file in list_test_files:
        test_file_name = test_file[:-4] # remove the txt extension
        test_file_path = 'data/' + test_file_name + '.txt'
        print(f"Currently sending test data from {test_file_name + '.txt'} ")
        
        if (pkg.activate_result_file_html):
            result_file_html = builder_html.init_html_result_file(test_file_name, test_file_path, css_file)

        if (pkg.activate_result_file_txt):
            result_file_txt = builder_txt.init_txt_result_file(test_file_name)
        
        last_rd_data = [] # keep in memory the last read data for cmp commands
        with open(test_file_path,"r") as test_file_opened:
            parsed_file = parser.parse_file(test_file_opened, result_file_html, result_file_txt)
            number_cmd_test_file = len(parsed_file)
            for indice_command_from_test_file in range(1,number_cmd_test_file+1):
                parsed_line = parsed_file[indice_command_from_test_file]
                last_rd_data = translator.command_disjunction(AxiQspi, result_file_html, result_file_txt, indice_command_from_test_file, parsed_line, last_rd_data)
                
        if (pkg.activate_result_file_html):
            builder_html.end_html_result_file(result_file_html)

        if (pkg.activate_result_file_txt):
            builder_txt.end_txt_result_file(result_file_txt)
                
        print(f"Finished sending test data from {test_file + '.txt'} ")
     
    
    
# SINGLE test config :
if (pkg.single_test_config["activate"]) :
    test_file_name = pkg.single_test_config['test_file'] 
    test_file_path = 'data/' + test_file_name + '.txt'
    print(f"Currently sending test data from {test_file_name + '.txt'} ")

    if (pkg.activate_result_file_html):
        result_file_html = builder_html.init_html_result_file(test_file_name, test_file_path, css_file)

    if (pkg.activate_result_file_txt):
        result_file_txt = builder_txt.init_txt_result_file(test_file_name)
    
    last_rd_data = [] # keep in memory th  e last read data for cmp commands
    with open(test_file_path,"r") as test_file_opened: 
        parsed_file = parser.parse_file(test_file_opened, result_file_html, result_file_txt)
        number_cmd_test_file = len(parsed_file)
        for indice_command_from_test_file in range(1,number_cmd_test_file+1):
            parsed_line = parsed_file[indice_command_from_test_file]
            last_rd_data = translator.command_disjunction(AxiQspi, result_file_html, result_file_txt, indice_command_from_test_file, parsed_line, last_rd_data)
            
    if (pkg.activate_result_file_html):
        builder_html.end_html_result_file(result_file_html)

    if (pkg.activate_result_file_txt):
        builder_txt.end_txt_result_file(result_file_txt)
            
    print(f"Finished sending test data from {test_file_name + '.txt'} ")
    

Configure device
Currently sending test data from my_test.txt 
 REGWR 404 FF 01   
 
REGWR (11) : B0 04 04 FF 00 00 00 01 00 00 00  
 
Command # 1: Finished Writing  

 REGWR 404 FF 03   
 
REGWR (11) : B0 04 04 FF 00 00 00 03 00 00 00  
 
Command # 2: Finished Writing  

 REGWR 404 FF 0F   
 
REGWR (11) : B0 04 04 FF 00 00 00 0F 00 00 00  
 
Command # 3: Finished Writing  

 REGWR 0 654321FF CBA9871F   
 
REGWR (11) : B0 00 00 FF 21 43 65 1F 87 A9 CB  
 
Command # 4: Finished Writing  

 REGRD 0   
 
REGRD (3) : A0 00 00  
 
Command # 5: Finished Reading; read value = 0000000F   

 REGCMP 0000000F 0000000F   
 
Comparison of read data and reference data 

 **** Command #%4d: Last Reg read value (masked) = 0F
    is equal to compared value =      0F **** 
 
The compared data are the same : SUCCESSFUL :
        the ref data is : 0000000F  
        the read byte is : 0000000F  
 

 REGWR 0 0000000F 00000000   
 
REGWR (11) : B0 00 00 0F 00 00 00 00 00 00 00  
 
Command # 7: Finished Writ

In [8]:
# Show clocks frequency
# from pynq import Clocks
# print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')
# print(f'FCLK1: {Clocks.fclk1_mhz:.6f}MHz')